<a href="https://colab.research.google.com/github/linccolnevans/datascience/blob/main/LINCEVANUnit8Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Your file on your github needs to be a a .ipynb file with the first four letters of one first name, the first four letters of one last name, and U6proj.

Ex1.

A student named Avy Aguacate Avocado Palta will submit a file named "Avy_AvocU8proj.ipynb", since she normally just goes by Avy, and, when pressed, reports Avocado as her last name. The " _ " after "Avy" serves as the fourth "letter" in her first name.

Ex2.

Army Armadillo will submit a file named "ArmyArmaU8proj.ipynb"

# Unit 8 Project: Writing a BARTical

**Task**:

Two parts:

1. Propose a data set that's compatible with a pymc_bart model. I must approve your submission before you move on to part 2.

2. Create, analyze, and present BART model, using pymc_bart within the colab environment, on a data set of your choice, with the following conditions:
  - your final model must have more than 2 predictors. The intercept term doesn't count as a predictor here (even though technically it is)
  - You must argue why your variable selection is justified. You may use either a theoretical or numerical argument. Additionally, you must comment on any potential interactions between your predictor variables.



Guidlines:

- Your data or your model type must be different from any project you've previously done.

- You are allowed to copy and paste any code from the notes, including my answer key (AK), without any reference or citations. Same goes for the course textbook, BAP3.

- This is an individual project. It is fine if you model the same process as a peer, but your raw data **must** be substantially different. "Substantially different" means that I, your instructor, arbitrarily think your data sets are different enough.

- Do not look at other student's code/jupyter notebooks. Copying, or copying and pasting, other students code will result in a failing grade. Also, I will tell all the other teachers I know that you cheated.

- If you want to know what another student did, ask them--any verbal communication about a project is allowed and encouraged. If you want to see a graphic they made, ask them to sketch a picture of it for you by hand.

- Any handwritten communication is also allowed and encouraged, so long as all parties are in the same physical location together. Example: you may sketch out a piece of code while verbally explaining to someone what you did. Do NOT: hand write the code for your entire project, take a picture, and send it to everyone in the course.

- If you use someone's code from outside this course (like from the internet, an AI, or your parent), you must ask me if it is ok, and you must cite them in your work. I don't care all they did was to click your space bar one more time: you tell me, and you cite their contribution. You don't need to tell me about or cite verbal or handwritten communication though.

- You may use a more complicated model--like something from further along in the course--IF I give you permission.

- If you are unsure whether or not something is ok or allowed: ASK BEFORE YOU DO IT.

Faliure to adhere to these guidlines will result in a failing grade.

Tips:

- Try to pick something that you care about, have knowledge of, experience with, or that you find interesting/cool.

- Gather data *before* picking a model or likelihood.

-  Use your best idea now; maybe you can look at the context you used in the previous project, and gather different data from it.

- Choose a data set with lots of columns, and lots of potential interactions between variables. It'll be more fun, and will make model/variable selection more relevant.

- Comment heavily, and explain your steps and thought process with text blocks. This is not only requried, it'll also help you understand what you're doing and what you want to do.

- Its ok to pick a data set for which you don't qute understand the context--so long as you think the context is cool.


In [30]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import seaborn as sns

In [31]:
!pip install pymc-bart
import pymc_bart as pmb

In [32]:
from sklearn import tree

In [33]:
bball = pd.read_csv("https://raw.githubusercontent.com/thedarredondo/data-science-fundamentals/refs/heads/main/Data/basketball2324.csv").dropna()
#drops all the non-numerical columns
bball.drop(columns=["Player","Pos","Tm","Player-additional"],inplace=True)
bball.query("MP > 2700", inplace=True)
bball.reset_index(drop=True,inplace=True)

In [34]:
shoot = bball[["FG%","3PA","FT%"]]

In [35]:
shoot

,FG%,3PA,FT%
0,0.455,351,0.725
1,0.436,592,0.814
2,0.479,526,0.847
3,0.480,225,0.853
4,0.523,407,0.856
5,0.461,532,0.836
6,0.434,258,0.791
7,0.583,231,0.817
8,0.459,554,0.794
9,0.594,87,0.704


In [36]:
#convert to numpy array
s_matrix = shoot.to_numpy()
#convert to a PyTorch tensor
s_tensor = torch.from_numpy(s_matrix)

NameError: name 'torch' is not defined

In [ ]:
s_tensor #the default data type is a float

In [ ]:
#the dimensions of our tensor/matrix--this will be of vital importance.
s_tensor.shape

In [ ]:
torch.matmul(s_tensor,s_tensor) #this will return a shape error

In [ ]:
SxSt = torch.matmul(s_tensor,s_tensor.T) #this will actually mutlipy our matrices

In [ ]:
SxSt,SxSt.shape

In [ ]:
bball = pd.read_csv("https://raw.githubusercontent.com/thedarredondo/data-science-fundamentals/refs/heads/main/Data/basketball2324.csv").dropna()
#drops all the non-numerical columns
bball.drop(columns=["Player","Pos","Tm","Player-additional"],inplace=True)
bball.query("MP > 2000", inplace=True) #still a select group of players
bball.reset_index(drop=True,inplace=True)
shoot = bball[["FG%","3PA","FT%"]] #only the three shooting metrics we're interested in

In [ ]:
#convert to numpy array
s_matrix = shoot.to_numpy()
#convert to a PyTorch tensor
s_tensor = torch.from_numpy(s_matrix)

In [ ]:
s_tensor

In [ ]:
#copied from pytorch fundamentals' online textbook

#this function defaults to a random  offset vector and weight matrix
linear = torch.nn.Linear(in_features=2, # in_features: must match column number of input
                         out_features=1,# out_features: column number of output
                         dtype=torch.float64) #specifies the data type

X = s_tensor[:,[1,2]] #use 3PA and FT% to predict
y_pred = linear(X) #the actual matrix multiplication
print(f"Input shape: {X.shape}\n")
print(f"Output:\n{y_pred}\n\nOutput shape: {y_pred.shape}")

In [ ]:
X = s_tensor[:,2].unsqueeze(dim=1) #use FT% as a predictor
y = s_tensor[:,0].unsqueeze(dim=1) #FG% is the estimand, the thing we're predicting.

# 80% of data is training, 20% is testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
#this function defaults to a random  offset vector and weight matrix
linear = torch.nn.Linear(
    in_features=1, # in_features: must match number of columns of input, which is now one
    out_features=1,# out_features: number of columns in output
    dtype=torch.float64) #specifies the data type

In [ ]:
# Create a loss function
loss_fn = nn.MSELoss() # mean squared error; this is similar to multiplying our matrix by itself

# Create an optimizer; SGD is stochastic gradient decent
optimizer = torch.optim.SGD(params=linear.parameters(),
                            lr=0.1) #this is how fast it optimizes; smaller is slower, but more consistent

In [ ]:
# Set the number of epochs; this is how many times we update our model
epochs = 10

for epoch in range(epochs):
    ### Training

    #this allows our matrices to update
    linear.train()

    # 1. Forward pass; makes predictions
    y_pred = linear(X_train)

    # 2. Calculate loss; how bad were our predictions?
    loss = loss_fn(y_pred, y_train)

    # 3. Optimizer zero grad; resets how we change our model
    optimizer.zero_grad()

    # 4. Loss backwards; determines how to change our weights matrix
    loss.backward()

    # 5. Optimizer step; changes our weights matrix based on .backward()
    optimizer.step()

    ### Testing
    linear.eval() #forces our matrices to stay the same
    with torch.inference_mode():
      # 1. Forward pass; makes predictions, but with test data
      test_pred = linear(X_test)
      # 2. Calculate the loss, but with test data
      test_loss = loss_fn(test_pred, y_test)

    # Print out what's happening
    if epoch % 1 == 0: #prints every 100 epochs
        print(f"Epoch: {epoch} | Train loss: {loss:.5f}, Test loss: {test_loss:.5f}")

In [ ]:
def plot_predictions(train_data=X_train,
                     train_labels=y_train,
                     test_data=X_test,
                     test_labels=y_test,
                     predictions=None):
  """
  Plots training data, test data and compares predictions.
  """

  # Plot training data in blue
  plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")

  # Plot test data in green
  plt.scatter(test_data, test_labels, c="g", s=4, label="Testing data")

  if predictions is not None:
    # Plot the predictions in red (predictions were made on the test data)
    plt.scatter(test_data, predictions, c="r", s=4, label="Predictions")

  # Show the legend
  plt.legend(prop={"size": 14});

In [ ]:
# Turn on evaluation mode
linear.eval()

# Make predictions (inference)
with torch.inference_mode():
    y_preds = linear(X_test)

# Plot data and predictions
plot_predictions(predictions=y_preds);

In [ ]:
howell = pd.read_csv("https://raw.githubusercontent.com/thedarredondo/data-science-fundamentals/refs/heads/main/Data/howell.csv")

#convert to a PyTorch tensor
h = howell.height.to_numpy()
height = torch.from_numpy(h)
w = howell.weight.to_numpy()
weight = torch.from_numpy(w)

In [ ]:
X = height.unsqueeze(dim=1) #use height as a predictor
y = weight.unsqueeze(dim=1) #weight is the estimand, the thing we're predicting.

In [ ]:
# Scale the height using standardization
scaler = StandardScaler()
height_scaled = scaler.fit_transform(
    height.reshape(-1, 1) #makes it a column vector
    )

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    height_scaled, weight, test_size=0.2,
)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

Start of the project, Ill be using basketball data and looked at unit 9 units to help inspire me so I added them above.

In [ ]:
basketball = pd.read_csv(
    'https://raw.githubusercontent.com/thedarredondo/data-science-fundamentals/main/Data/basketball2324.csv').dropna()

In [ ]:
#X is our predictor variable, upper beak measurment
X = basketball["FT%"].to_numpy().reshape(-1, 1)
#Y is the predicted variable, total beak weight
Y = basketball["3P%"].to_numpy()

In [ ]:
#fits a decision tree. This is analogous to running pm.sample,
#except it only finds the mean. Kinda like that, anyway.
dtree = tree.DecisionTreeRegressor(max_depth=2)
basketballtree = dtree.fit(X,Y)

#uses the fitted model to predict total weight for various
#unseen upper beark measurements
#This is analogous to a posterior predictive distribution,
#if we only found the posterior predictive mean...
#...kinda.
X_test = np.arange(75, 250, 1)[:, np.newaxis]
y = basketballtree.predict(X_test)

In [ ]:
tree.plot_tree(basketballtree)

In [ ]:
plt.figure()
plt.scatter(X, Y, s=20, edgecolor="black", c="darkorange", label="actual measurements")
plt.plot(X_test, y, color="cornflowerblue", linewidth=2)
plt.xlabel("FT%")
plt.ylabel("3P%")
plt.title("Decision Tree Regression")
plt.legend()
plt.show()

I really thought this would end up cooler and help me understand percentage to free throws but it literally doesnt work... free throw percentage just doesn't exist apparently despite being in the database.

My plan was to see the overall stat about 3s and freethrows, then divide it into positions and see how BART thinks people will shoot according to their positions and if it would have all centers shoot low percentages at both or if point guards or shooting guards would be higher on the list.